In [1]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append('/mnt/data1/spatial/tissue-alignment/traversal')
from io_utils import read_image, read_spatial_data

%matplotlib inline

section_path = f'/mnt/data1/spatial/data/colon/UC/B'

In [2]:
fullres = False
im = read_image(section_path, fullres)
pos_df, spot_radius = read_spatial_data(section_path, fullres)

In [3]:
sys.path.append('/mnt/data1/spatial/belayer/src')
from utils_IO import *

from harmonic import *
from region_cost_fun import *
from dprelated import *
from dp_post_processing import *
from general_helper_funcs import *
from precompute_likelihood import *
from dp_linear_boundary import *
from slideseq_helpers import *

In [4]:
# input_10x_folder='151673'
count, coords, barcodes, gene_labels = read_input_10xdirectory(section_path)

G, N=count.shape

# fill in gaps in 10x hexagonal grid
fullpoints, in_tissue = fill_geometry(coords, is_hexagon=True)

# get boundary points
boundary_idx_file='colon-UC-B-boundaries.npy'
bounds = np.load(boundary_idx_file, allow_pickle=True)
new_bounds = []
for b in bounds:
    new_bounds.append(pos_df.loc[b, ['row', 'col']].to_numpy())
new_bounds = np.asarray(new_bounds, dtype=object)
boundary_file = 'colon-UC-B-boundaries-coords.npy'
np.save(boundary_file, new_bounds)
boundary_list = read_boundary_list(boundary_file, fullpoints)

In [5]:
spos = spatialcoord(x = fullpoints[:,0], y = fullpoints[:,1])
har = harmonic(fullpoints, spos.adjacency_mat, np.sqrt(spos.pairwise_squared_dist))

interpolation = har.interpolation_using_list( boundary_list )
depth = interpolation[in_tissue]

In [6]:
pos_df.loc[pos_df['barcode'].argsort(), 'depth'] = depth

In [7]:
pos_df['depth2'] = pos_df['depth'].round().astype(int)
pos_df['depth2'] = pos_df['depth2'] - pos_df['depth2'].min() # make min 0

In [8]:
assert not (pos_df['depth2'].value_counts() == 0).any()

In [9]:
from graph import compute_path_counts
from io_utils import read_transcription_data

counts = read_transcription_data(section_path)

dummy_path_idxs = [-1] * (pos_df['depth2'].max() + 1) # assumes there are no gaps in depth2, see prior assert
mean_counts, std_counts = compute_path_counts(
    pos_df=pos_df,
    counts=counts,
    path_idxs=dummy_path_idxs,
    clusters=pos_df['depth2'],
)

QUEUEING TASKS | :   0%|          | 0/164 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/164 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/164 [00:00<?, ?it/s]

In [10]:
mean_counts.to_csv('belayer_uc_counts.csv', index=False)

In [11]:
pos_df[['depth', 'depth2']].to_csv('belayer_uc_depths.csv', index=False)

In [12]:
normalized_length = 101

xp = np.linspace(0, 1, len(mean_counts))
_mean_counts = mean_counts.dropna(axis=0)
xp = xp[_mean_counts.index.to_numpy()]
x = np.linspace(0, 1, normalized_length)
out = []
for i in range(_mean_counts.shape[1]):
    interp = np.interp(x, xp, _mean_counts.iloc[:, i].to_numpy())
    out.append(interp)
path_normalized_counts = pd.DataFrame(np.asarray(out).T, columns=mean_counts.columns)
path_normalized_counts.to_csv(f'belayer_uc_normalized_path_counts.csv', index=False)